In [ ]:
!pip install torch>=2.0.0 transformers>=4.31.0 sentence-transformers faiss-cpu PyMuPDF numpy accelerate 'bitsandbytes>=0.41.0'

In [1]:
#
import fitz
import torch
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig


In [2]:
import os
os.environ['HUGGINGFACE_TOKEN'] = 'your token here'

In [3]:
class SimpleRAG:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.documents = []
        self.doc_titles = []
        self.embed_model = None
        self.index = None
        self.tokenizer = None
        self.model = None
        # auto-detect GPU or CPU
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def load_pdfs(self):
        pdf_files = [f for f in os.listdir(self.directory_path) if f.endswith('.pdf')]
        for f in pdf_files:
            full_path = os.path.join(self.directory_path, f)
            text = self.extract_text_from_pdf(full_path)
            if text:
                self.documents.append(text)
                self.doc_titles.append(f)
        print(f"Loaded {len(self.documents)} PDFs.")

    def extract_text_from_pdf(self, pdf_path):
        doc_text = []
        try:
            doc = fitz.open(pdf_path)
            for page in doc:
                txt = page.get_text()
                if txt.strip():
                    doc_text.append(txt)
            doc.close()
        except Exception as e:
            print(f"Error reading {pdf_path} - {e}")
            return None
        return " ".join(doc_text)

    def init_embeddings(self):
        self.embed_model = SentenceTransformer('all-MiniLM-L6-v2')
        print("SentenceTransformer loaded.")

    def create_index(self):
        print("Creating FAISS index.")
        embeddings = self.embed_model.encode(self.documents, show_progress_bar=True)
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(np.array(embeddings).astype('float32'))
        print("Index created and docs embedded.")

    def init_llama(self):
        print("Loading Llama 2 7B in 4-bit.")
        model_name = "meta-llama/Llama-2-7b-chat-hf"
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=False,
            bnb_4bit_quant_type="nf4"
        )
        self.tokenizer = LlamaTokenizer.from_pretrained(model_name)
        self.model = LlamaForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            # Optional: set your cache_dir if needed
            cache_dir='/media/joepareti54/Elements/x/huggingface/hub/more/',
            torch_dtype=torch.float16,
            device_map="auto"
        )
        print("Llama 7B model loaded in 4-bit.")

    def retrieve_documents(self, query, k=3):
        print(f"\nQuery: {query}")
        query_embed = self.embed_model.encode([query])[0]
        distances, indices = self.index.search(np.array([query_embed]).astype('float32'), k)
        retrieved_docs = []
        for rank, idx in enumerate(indices[0]):
            dist = distances[0][rank]
            doc_text = self.documents[idx]
            doc_title = self.doc_titles[idx]
            print(f"  Rank {rank+1}, Distance={dist:.4f}, Doc={doc_title}")
            retrieved_docs.append(doc_text)
        return retrieved_docs

    def summarize_text(self, text: str) -> str:
        """
        Step 1: Summarize the doc text in a short prompt so we don't pass huge data to final generation.
        """
        # Possibly truncate text first if it's huge
        max_chars = 1000
        truncated_text = text[:max_chars]

        prompt = (
            "You are an AI summarizer. Summarize the following text in your own words. "
            "Do NOT repeat large chunks verbatim.\n\n"
            f"{truncated_text}\n\n"
            "Summary:"
        )
        input_ids = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids.to(self.device)
        outputs = self.model.generate(
            input_ids=input_ids,
            max_new_tokens=100,
            num_beams=1,
            do_sample=True,
            temperature=0.7
        )
        summary = self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        return summary

    def generate_final_answer(self, summary: str, query: str) -> str:
        """
        Step 2: Use the doc summary plus user query to produce a final answer.
        """
        prompt = (
            "You are a knowledgeable AI with broad expertise.\n"
            f"Here is a summary of a reference document:\n{summary}\n\n"
            f"User question: {query}\n\n"
            "Answer in your own words using the summary. Do not repeat the summary verbatim:"
        )
        input_ids = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).input_ids.to(self.device)
        outputs = self.model.generate(
            input_ids=input_ids,
            max_new_tokens=150,
            num_beams=1,
            do_sample=True,
            temperature=0.7
        )
        answer = self.tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        return answer

    def run_query(self, query, k=3):
        docs = self.retrieve_documents(query, k)
        if not docs:
            print("No docs returned. Possibly the index is empty.")
            return

        # Optionally combine or just pick the top doc
        # e.g., summarizing only the 1st doc for demonstration
        # or you can merge them if you want multiple summaries
        doc_text = docs[0]

        # Step 1: Summarize doc text
        doc_summary = self.summarize_text(doc_text)

        # Step 2: Generate final answer
        final_answer = self.generate_final_answer(doc_summary, query)
        print(f"\nAnswer:\n{final_answer}")




In [ ]:
def main():
    directory_path = "/media/joepareti54/Elements/x/All_Finance_PDF_files_old/"
# update the line above using your own storage
    rag = SimpleRAG(directory_path)
    rag.load_pdfs()
    rag.init_embeddings()
    rag.create_index()
    rag.init_llama()

    while True:
        query = input("\nEnter query (or 'quit'): ").strip()
        if query.lower() == 'quit':
            break
        rag.run_query(query, k=3)

if __name__ == "__main__":
    main()


Loaded 2665 PDFs.
SentenceTransformer loaded.
Creating FAISS index.


Batches:   0%|          | 0/84 [00:00<?, ?it/s]

Index created and docs embedded.
Loading Llama 2 7B in 4-bit.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Llama 7B model loaded in 4-bit.


...


Query: is 5g an important techology for car manufacturing ?
  Rank 1, Distance=0.8409, Doc=5G BlogFinal-0x.pdf
  Rank 2, Distance=0.9388, Doc=Everything You Need to Know About 5G - WSJ.pdf
  Rank 3, Distance=0.9529, Doc=WP_5G_for_Connected_Industries_and_Automation_Download_19.03.19.pdf

Answer:
You are a knowledgeable AI with broad expertise.
Here is a summary of a reference document:
You are an AI summarizer. Summarize the following text in your own words. Do NOT repeat large chunks verbatim.

 
1 
 
5G Technologies in the Manufacturing Industry: Opportunities and 
Challenges 
Wednesday, February 19, 2020  joepareti54@mail.com 
Contents 
5G Technologies in the Manufacturing Industry: Opportunities and Challenges ........................................ 1 
Key Points .................................................................................................................................................. 1 
The role of Software and Standardization to reap the 5G benefits......


Enter query (or 'quit'):  what are important technologies to accelerate product development



Query: what are important technologies to accelerate product development
  Rank 1, Distance=1.0854, Doc=‘Predictive-Maintenance’ Tech Is Taking Off as Manufacturers Seek More Efficiency - WSJ.pdf
  Rank 2, Distance=1.1008, Doc=Mayo Clinic, Others Use ‘AI Factories’ to Speed AI Development.pdf
  Rank 3, Distance=1.1131, Doc=New Research Busts Popular Myths About Innovation - WSJ.pdf

Answer:
You are a knowledgeable AI with broad expertise.
Here is a summary of a reference document:
You are an AI summarizer. Summarize the following text in your own words. Do NOT repeat large chunks verbatim.

Startups that make technology designed to predict industrial equipment failures before
they happen are seeing a surge in demand, as strained supply chains prompt
manufacturers to squeeze more efficiency out of production lines, startup founders and
analysts say.
Anna Farberov,
general manager of
PepsiCo
Labs, the technology venture arm of PepsiCo
Inc., said that over the past year so-called predict


Enter query (or 'quit'):  are humans an evolution step of apes or of fish



Query: are humans an evolution step of apes or of fish
  Rank 1, Distance=1.4519, Doc=Startup Fauna Bio Studies Animal Genomes for Clues to Human Diseases - WSJ.pdf
  Rank 2, Distance=1.5220, Doc=Learning from evolution_ Using AI language models to design proteins.pdf
  Rank 3, Distance=1.5617, Doc=Google Debuts Cybersecurity-Focused AI System.pdf

Answer:
You are a knowledgeable AI with broad expertise.
Here is a summary of a reference document:
You are an AI summarizer. Summarize the following text in your own words. Do NOT repeat large chunks verbatim.

This copy is for your personal, non-commercial use only. Distribution and use of this material are governed by our Subscriber Agreement and by copyright law. For
non-personal use or to order multiple copies, please contact Dow Jones Reprints at 1-800-843-0008 or visit www.djreprints.com.
https://www.wsj.com/articles/startup-fauna-bio-studies-animal-genomes-for-clues-to-human-diseases-cf5f86dc
INDUSTRY NEWS
Startup Fauna Bio Studies 